# Predict Product Prices

Training



In [ ]:
# pip installs
# trl (trandformer reinformecent learning) whihc conians sft supevised fine tunning trainer

!pip install -q datasets==2.21.0 requests torch peft bitsandbytes transformers==4.43.1 trl accelerate sentencepiece wandb matplotlib

In [ ]:
# imports
# With much thanks to Islam S. for identifying that there was a missing import!

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Constants
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer-aws-desc"
HF_USER = "Meziane"

# Data
#DATASET_NAME = f"{HF_USER}/pricer-data"
DATASET_NAME = "ed-donner/pricer-data"

#  the data is always cratedfted to do not be more then 179, adding on a few token for the start of sentece for the tokneizer
# this means that we are going to say maximus sequrnce lenght of 182 and this is very importamt becasuie evry traoing
# data point it will fitted into this so that and has to be small enoguh to fit in gpu's memory that way I have evry data point will be fitted into this
# so tahat and tis needs ot be a small enoguh tnumber fto fit in our gpu memory thats why i have cut of the the desccpiton to fit it in out gpu memory
MAX_SEQUENCE_LENGTH = 182 #



# Run name for saving the model in the hub
# Hugging Face models are Git repositories, so I can push new versions of a model to the same repo.
# However, I prefer to keep separate repositories for each training run. This allows me to organize models
# by different hyperparameters or settings, with versioning (e.g., different epochs) within each repo.
# It's not required but helps maintain clarity and track changes effectively.
# Example: Start with a unique run name for the model.

RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"




# --- Hyperparameters for Model Training ---



LORA_R = 8  # The rank of the low-rank adaptation matrices.
# A lower rank (smaller value) uses less memory but can reduce performance.
# A higher rank (larger value) allows more complex adaptations and can potentially improve performance.
# A good balance is usually between 4-16 for most tasks.

LORA_ALPHA = 16  # Scaling factor for the low-rank adaptation (LORA).
# Higher values (e.g., 16, 32) make the LORA modifications more impactful, effectively strengthening the model's capacity to adapt.
# Lower values might make the modifications less prominent.

TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]  # List of model modules (like query, key, value projections) to apply LORA.
# These modules are the core components in Transformer models (especially attention layers).
# Adjusting these allows us to fine-tune the attention mechanisms without modifying the entire network.

LORA_DROPOUT = 0.1  # Dropout rate to apply during training.
# Dropout is a regularization technique to prevent overfitting by randomly "dropping out" a percentage of neurons during training.
# A value of 0.1 means 10% of the neurons will be randomly ignored during training, which can help improve generalization.

QUANT_4_BIT = True  # Whether to apply 4-bit quantization to the model weights for efficiency.
# Quantization reduces model size and speeds up inference by using lower precision arithmetic.
# However, it sacrifices some precision in exchange for reduced memory footprint and faster computations.
# 4-bit quantization is often used for deploying models in environments with memory limitations.





# --- Training Configuration ---



EPOCHS = 3  # The number of full passes through the training dataset.
# Training over more epochs usually helps the model converge to a better solution, but too many epochs can lead to overfitting.
# In practice, values like 3-5 epochs are common, but this can depend on the dataset and task complexity.

BATCH_SIZE = 16  # The number of training samples used in one forward/backward pass.
# Larger batch sizes can improve training stability and efficiency, but they also require more GPU memory.
# Depending on GPU capacity, you may want to adjust the batch size for optimal memory utilization.
# Typically, batch sizes like 16, 32, or 64 are used in deep learning tasks.

GRADIENT_ACCUMULATION_STEPS = 1  # Number of steps to accumulate gradients before performing an optimization step.
# Gradient accumulation allows you to simulate larger batch sizes without needing the memory to process them all at once.
# For instance, if you set it to 4, gradients will accumulate for 4 steps before updating the model weights, simulating a batch size 4 times larger.

LEARNING_RATE = 1e-4  # The learning rate determines how big each update to the model's weights will be.
# A learning rate that is too high can cause the model to overshoot the optimal solution, while too low a rate may result in slow convergence.
# A learning rate of 1e-4 is a common starting point for many deep learning models, though experimentation may be needed for your specific task.

LR_SCHEDULER_TYPE = 'cosine'  # The type of learning rate scheduler to use during training.
# 'Cosine' learning rate decay decreases the learning rate gradually as training progresses, making the training process more stable in the later stages.
# Other options like 'linear' or 'step' can also be used, but cosine decay often works well for transformer-based models.

WARMUP_RATIO = 0.03  # The proportion of total training steps during which the learning rate gradually increases.
# This allows the model to start with a lower learning rate and avoid unstable updates, especially in the first stages of training.
# A value of 0.03 means that the learning rate will ramp up for 3% of the total training steps before reaching the target value.

# --- Optimizer Settings ---
OPTIMIZER = "paged_adamw_32bit"  # The optimizer used for training. "AdamW" is a version of Adam that includes weight decay.
# AdamW is a well-known optimizer that works well for transformers and large-scale deep learning tasks.
# "Paged" AdamW refers to a memory-efficient variant of AdamW that allows for better handling of large models by storing gradients in memory more efficiently.
# This optimizer supports 32-bit precision, which is important for large models like transformers.



# --- Administrative Configuration ---



STEPS = 50  # The number of steps (batches) to process before logging training progress or saving checkpoints.
# Logging every few steps helps monitor training performance and track any issues during training.
# This is useful for tracking metrics such as loss, accuracy, etc.

SAVE_STEPS = 5000  # How often the model's weights should be saved during training.
# Regularly saving the model allows you to resume training in case of interruptions, or to track the best performing model.
# Saving every 5000 steps is typical, though this can vary depending on the dataset size and training duration.

LOG_TO_WANDB = True  # Whether to log training metrics to Weights and Biases (WandB).
# WandB provides a platform for tracking, visualizing, and comparing machine learning experiments in real-time.
# Logging to WandB can give you insights into training curves, hyperparameter settings, and model performance, which can be useful for debugging and optimization.



%matplotlib inline

In [ ]:
HUB_MODEL_NAME

'Meziane/pricer-aws-desc-2024-11-30_12.34.42'

-More on Optimizers

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice

The most common is Adam or AdamW (Adam with Weight Decay).  
Adam achieves good convergence by storing the rolling average of the previous gradients; however, it adds an additional memory footprint of the order of the number of model parameters.


In [17]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [11]:
# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
# Setting up additional Weights & Biases configurations:
os.environ["WANDB_LOG_MODEL"] = "true" if LOG_TO_WANDB else "false"  # Controls model logging.
os.environ["WANDB_WATCH"] = "gradients"  # Enables gradient logging during training.


In [12]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

Generating train split:   0%|          | 0/400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
test[0]

{'text': "How much does this cost to the nearest dollar?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K\n\nPrice is $",
 'price': 374.41}

In [ ]:

# Note: This line is not strictly required. Hugging Face's Trainer will automatically handle
# Weights & Biases integration if you start the run without setting this environment variable.
# However, I sometimes include it explicitly to have more control over the project setup and configuration.

# You can also pass a config dictionary with parameters to customize the logging behavior further.
# This lets you add extra attributes that will appear in the Weights & Biases dashboard.
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Load the Tokenizer and Model

The model is "quantized" -  reducing the precision to 4 bits.

In [14]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)


# The purpose of this code is to ensure that each data point in the dataset is padded to the maximum sequence length.
# The padding is done using the 'end of sentence' (EOS) token to fill up the sequence length to the expected size,
# which in this case is 182 tokens. This ensures all input sequences are the same length.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)



# This step avoids unnecessary warnings during training related to missing pad token ID in generation config.
# Though it might seem redundant, this setting ensures that no warning is raised during generation steps.
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

# Data Collator


In [ ]:
# The goal here is to focus the model's training on predicting the price token after the dollar sign ($),
# rather than the full prompt or description. We don't want the model to focus on learning the prompt,
# but instead, we want it to learn how to predict the price immediately following the dollar sign.

# To achieve this, we set up a mask that tells the model to ignore learning about the entire prompt (e.g., product description)
# and instead, focus on predicting the token that follows the "Price is $" part of the input text.

# Setting up masks for specific tokens in the input text can be tricky, but Hugging Face provides a simple solution
# through the `DataCollatorForCompletionOnlyLM`. This utility is specifically designed for situations where you want
# the model to complete the sentence starting from a specific token, without needing it to learn about the rest of the input.

from trl import DataCollatorForCompletionOnlyLM

# Define the response template. This tells the model what part of the input text we want to focus on.
# In this case, we are providing a simple prompt: "Price is $", and we want the model to predict what follows it.
response_template = "Price is $"

# Create a data collator object that will handle the masking automatically, so the model learns to predict the price.
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)




## Setting up the configuration for Training

 Create 2 objects:

A LoraConfig object with our hyperparameters for LoRA

An SFTConfig with our overall Training parameters

In [ ]:
# First, specify the configuration parameters for LoRA

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",# a good practice is that to use eval strategy but since my datset is big avoid to do.
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    tokenizer=tokenizer,
    args=train_parameters,
    data_collator=collator
)

In [ ]:
# Fine-tunning
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

In [ ]:
if LOG_TO_WANDB:
  wandb.finish()